In [44]:
import pandas as pd

In [45]:
df_train = pd.read_csv("train titanic.csv")
df_test = pd.read_csv("test titanic.csv")

In [46]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


**Extracting titles from name and selecting first four only**

In [47]:
titles = df_train['Name'].apply(lambda X:X.split(' ')[1]).value_counts()[:4].index

**Creating New Column named Title**

In [48]:
df_train['Title'] = df_train['Name'].apply(lambda X:X.split(' ')[1])
df_test['Title'] = df_test['Name'].apply(lambda X:X.split(' ')[1])

# If Title is not in titles then assign it to the other

df_train['Title'] = df_train['Title'].apply(lambda X: X if X in titles else 'other')
df_test['Title'] = df_test['Title'].apply(lambda X: X if X in titles else 'other')

**EDA Dividing Data in Numeric and object Data**

In [49]:
columns_object = list(df_train.select_dtypes(include='object').columns)
columns_numeric = list(df_train.select_dtypes(exclude='object').columns)
columns_numeric, columns_object

(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'],
 ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Title'])

In [50]:
# Remove Columns which we are not going to handle  and use remaining

columns_numeric = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
columns_object = ['Sex', 'Embarked', 'Title']

**Handling Missing Data**

In [51]:
from sklearn.impute import SimpleImputer

In [52]:
imputer_numeric = SimpleImputer(strategy='median')
imputer_object = SimpleImputer(strategy='most_frequent')
imputer_numeric.fit(df_train[columns_numeric])
imputer_object.fit(df_train[columns_object])

df_train[columns_numeric] = imputer_numeric.transform(df_train[columns_numeric])
df_train[columns_object] = imputer_object.transform(df_train[columns_object])

In [54]:
df_test[columns_numeric] = imputer_numeric.transform(df_test[columns_numeric])
df_test[columns_object] = imputer_object.transform(df_test[columns_object])

In [55]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,S,Mr.
1,2,1,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,C,Mrs.
2,3,1,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,NaN,S,Miss.
3,4,1,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S,Mrs.
4,5,0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0.0,373450,8.0500,NaN,S,Mr.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2.0,"Montvila, Rev. Juozas",male,27.0,0.0,0.0,211536,13.0000,NaN,S,other
887,888,1,1.0,"Graham, Miss. Margaret Edith",female,19.0,0.0,0.0,112053,30.0000,B42,S,Miss.
888,889,0,3.0,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1.0,2.0,W./C. 6607,23.4500,NaN,S,Miss.
889,890,1,1.0,"Behr, Mr. Karl Howell",male,26.0,0.0,0.0,111369,30.0000,C148,C,Mr.


In [12]:
df_train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Title            0
dtype: int64

In [13]:
df_test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
Title            0
dtype: int64

**Binning Age**

In [56]:
def get_age_group(X):
    if X>50:
        return 'older'
    if X>25:
        return 'young'
    return 'kid'

In [57]:
# BINNING 
df_train['Age_group'] = df_train['Age'].apply(get_age_group)
df_test['Age_group'] = df_test['Age'].apply(get_age_group)

In [58]:
columns_object.append('Age_group')


In [59]:
columns_object

['Sex', 'Embarked', 'Title', 'Age_group']

**Feature Encoding**



In [60]:
# Feature Encoding
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore') # handle unknown used to handle data if unkown data occur in testing 
encoder.fit(df_train[columns_object])

OneHotEncoder(handle_unknown='ignore')

In [61]:
# Working on Train data
temp = encoder.transform(df_train[columns_object]).toarray()
df_temp = pd.DataFrame(temp)  #  Creating DF_temp of OneHot Encoded data
df_train = pd.concat([df_train,df_temp], axis=1) # Concating df_train,df_temp 
columns_numeric.extend(list(df_temp.columns)) # adding df_temp column in column numeric 

In [20]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,3,4,5,6,7,8,9,10,11,12
0,1,0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,2,1,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,3,1,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,1,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,5,0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0.0,373450,8.0500,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2.0,"Montvila, Rev. Juozas",male,27.0,0.0,0.0,211536,13.0000,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
887,888,1,1.0,"Graham, Miss. Margaret Edith",female,19.0,0.0,0.0,112053,30.0000,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
888,889,0,3.0,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1.0,2.0,W./C. 6607,23.4500,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
889,890,1,1.0,"Behr, Mr. Karl Howell",male,26.0,0.0,0.0,111369,30.0000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [62]:
#Working on test data
temp = encoder.transform(df_test[columns_object]).toarray()
df_temp = pd.DataFrame(temp)   #  Creating DF_temp of OneHot Encoded data
df_test = pd.concat([df_test,df_temp], axis=1)# Concating df_test,df_temp 

In [27]:
# columns_numeric.remove('Age')  This makes Score reduced by 0.48 hence commented

In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [65]:
p = {'n_estimators':[25,50,100,150,200], 'max_depth':[2,3,4,5,6], 'min_samples_leaf':[3,4,5,6,7]}
grid_cv = GridSearchCV(RandomForestClassifier(random_state=2),p,n_jobs=-1,cv=5,)
grid_cv.fit(df_train[columns_numeric], df_train['Survived']) #fit(X,y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2), n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_leaf': [3, 4, 5, 6, 7],
                         'n_estimators': [25, 50, 100, 150, 200]})

In [66]:
grid_cv.best_estimator_

RandomForestClassifier(max_depth=6, min_samples_leaf=4, n_estimators=150,
                       random_state=2)

In [67]:
grid_cv.best_params_

{'max_depth': 6, 'min_samples_leaf': 4, 'n_estimators': 150}

In [72]:
# model_final = RandomForestClassifier(max_depth=6, min_samples_leaf=4, n_estimators=150,random_state=2)
model_final = RandomForestClassifier(max_depth=5, min_samples_leaf=4, random_state=1, n_estimators=150)
# model_final = RandomForestClassifier(max_depth=5, min_samples_leaf=4, random_state=2, n_estimators=50)
# model_final = RandomForestClassifier(n_estimators=1000, max_depth=3, random_state=2)
model_final.fit(df_train[columns_numeric], df_train['Survived'])

RandomForestClassifier(max_depth=5, min_samples_leaf=4, n_estimators=150,
                       random_state=1)

In [73]:
y = model_final.predict(df_test[columns_numeric])

In [74]:
df_test['Survived'] = y

In [75]:
df_test[['PassengerId','Survived']].to_csv('sub7_dsml14.csv',index=False)